In [3]:
#FineMuay Client
#2023  Made by Camilo Lozano
###Install the following libs
#!pip install imutils imagezmq
#!pip install pyngrok
#!pip install opencv-python
#!pip install opencv_jupyter_ui

This client is part of [FineMuay](https://github.com/archoldkami/FineMuay) project 

This proyect uses [NGROK](https://ngrok.com/) and [imagezmq](https://github.com/jeffbass/imagezmq)

Create a NGROK account to send images to a public internet address

Set the parameters in the next cell.

In [5]:
#the NGROK TOKEN here:
NGROK_AUTH_TOKEN = ""
#set the FPS here, 10-16 range is a realistic measure at 640x480 resolution:
FPS =16
# set SRC= 0 for webcam source
SRC = 0
#set the jpg compression
jpg_compression =80

In [6]:
#FineMuay client uses imagezmp pub/sub mode.

import sys
import time
import numpy as np
import cv2
import imagezmq
import imutils
from imutils.video import VideoStream
import zmq
import ipywidgets as widgets
from IPython.display import display
import traceback
import opencv_jupyter_ui as jcv2
from pyngrok import ngrok
import datetime

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(5555, "tcp")
print("Set this address on Colab or your server:", ngrok_tunnel.public_url)
print("Sending at 640x480 --  Started at:" + str(datetime.datetime.now()))
print("Click the 'q' key to quit")
display_handle=display("Warming", display_id=True)
rpi_name = "finemuay_test"
res_dim = 640,480
picam = VideoStream(src=SRC,resolution=res_dim,framerate=FPS).start()
time.sleep(2.0)  # allow camera sensor to warm up
#set the client as pub/sub sender
sender = imagezmq.ImageSender(connect_to='tcp://*:5555', REQ_REP=False)
jcv2.setKeys (['q','space'])
i =0
start_time = time.time()
wait_time=round((1/FPS)*825,0)
try:
    while True:
        image = picam.read()
        i+=1
        image = imutils.resize(image, width=640)
        ret_code, jpg_buffer = cv2.imencode(".jpg", image, [int(cv2.IMWRITE_JPEG_QUALITY), jpg_compression]) 
        # send to the message queue server
        sender.send_jpg(rpi_name + str(i), jpg_buffer)
        display_handle.update("sending at FPS:{} Frame:{}".format(round(1.0 / (time.time() - start_time),2),i))
        start_time = time.time()
        #this waits enough time to complete the required FPS
        if jcv2.waitKey(wait_time)==ord('q'):
            break

except Exception as ex:
        print('Python error with no Exception handler:')
        print('Traceback error:', ex)
        traceback.print_exc()
finally:
    picam.stop()
    picam=None
    sender.close()
    sender=None
    ngrok.kill()
    jcv2.destroyAllWindows()
    print("Done !!!  " + str(datetime.datetime.now()))



Set this address on Colab or your server: tcp://4.tcp.ngrok.io:11145
Sending at 640x480 --  Started at:2024-01-17 11:42:41.008393
Click the 'q' key to quit


'sending at FPS:27.78 Frame:299'

Done !!!  2024-01-17 11:42:54.839221
